# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [1]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [2]:
print(sinasc.drop_duplicates().shape)
# ou
unicos = sinasc.drop_duplicates()
print(unicos.shape)

(27028, 69)
(27028, 69)


In [3]:
total = sinasc.shape[0]
unicos = sinasc.drop_duplicates().shape[0]
print(total, unicos, total > unicos)

27028 27028 False


In [5]:
# 2) seu código aqui

# já carregado por você
sinasc = pd.read_csv('SINASC_RO_2019.csv')

# número total de linhas (útil para a porcentagem)
n_total = sinasc.shape[0]

# contagem absoluta de missing por coluna
missing_count = sinasc.isna().sum()

# porcentagem de missing por coluna
missing_pct = (missing_count / n_total * 100).round(2)

# DataFrame resumo ordenado do maior para o menor missing
resumo_missing = pd.DataFrame({
    "missing_count": missing_count,
    "missing_percent": missing_pct
}).sort_values(by="missing_count", ascending=False)

# mostrar na tela as top 20 colunas com mais missing
print(resumo_missing.head(20))

# se quiser salvar o resumo em CSV no disco
resumo_missing.to_csv("resumo_missing_sinasc_ro_2019.csv", index=True)


            missing_count  missing_percent
DTRECORIGA          27028           100.00
CODANOMAL           26814            99.21
IDADEPAI            19421            71.86
SERIESCMAE          12710            47.03
DTULTMENST          10072            37.27
CODOCUPMAE           2907            10.76
MESPRENAT            2867            10.61
QTDFILMORT           2098             7.76
QTDPARTNOR           1879             6.95
QTDPARTCES           1791             6.63
QTDFILVIVO           1573             5.82
SEMAGESTAC           1238             4.58
TPMETESTIM           1238             4.58
GESTACAO             1232             4.56
QTDGESTANT           1212             4.48
STTRABPART            947             3.50
CONSPRENAT            930             3.44
STCESPARTO            747             2.76
RACACORMAE            661             2.45
RACACOR               647             2.39


In [6]:
# 3) seu código aqui

cols = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
        'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']

# selecionar apenas as colunas de interesse (evita erro se alguma coluna não existir)
selecionadas = [c for c in cols if c in sinasc.columns]
df_sel = sinasc[selecionadas].copy()

# total de linhas para calcular porcentagem
n_total = df_sel.shape[0]

# contagem absoluta e porcentagem de missing
missing_count = df_sel.isna().sum()
missing_pct = (missing_count / n_total * 100).round(2)

# resumo ordenado (maior para menor missing)
resumo_missing = pd.DataFrame({
    "missing_count": missing_count,
    "missing_percent": missing_pct
}).sort_values(by="missing_count", ascending=False)

print(resumo_missing)

# opcional: salvar para inspeção posterior
resumo_missing.to_csv("resumo_missing_selecionadas_sinasc_ro_2019.csv", index=True)


            missing_count  missing_percent
QTDFILVIVO           1573             5.82
GESTACAO             1232             4.56
ESTCIVMAE             317             1.17
ESCMAE                312             1.15
APGAR5                103             0.38
GRAVIDEZ               79             0.29
LOCNASC                 0             0.00
IDADEMAE                0             0.00
CONSULTAS               0             0.00


In [7]:
# 4) seu código aqui
apgar_col = "APGAR5"
if apgar_col not in sinasc.columns and "APGARS" in sinasc.columns:
    apgar_col = "APGARS"

# colunas de interesse
cols = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
        'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', apgar_col]

# selecionar colunas que existem no DataFrame
selecionadas = [c for c in cols if c in sinasc.columns]
df_sel = sinasc[selecionadas].copy()

# tratar strings vazias e espaços como NA
df_sel = df_sel.replace(r'^\s*$', pd.NA, regex=True)

# (opcional) tratar sentinelas numéricas comuns como NA — ajuste conforme necessário
sentinelas = { 'IDADEMAE': {9: pd.NA, 99: pd.NA}, 'CONSULTAS': {9: pd.NA}, 'QTDFILVIVO': {9: pd.NA, 99: pd.NA} }
for col, mapping in sentinelas.items():
    if col in df_sel.columns:
        df_sel[col] = df_sel[col].replace(mapping)

# contagens antes da remoção
total_antes = df_sel.shape[0]
missing_antes = df_sel.isna().sum()

# remover registros com Apgar5 ausente
df_filtrado = df_sel.dropna(subset=[apgar_col]).copy()

# contagens depois da remoção
total_depois = df_filtrado.shape[0]
missing_depois = df_filtrado.isna().sum()

# mostrar resultados
print("Coluna usada para Apgar:", apgar_col)
print("Total de registros (antes):", total_antes)
print("Total de registros (depois de remover Apgar faltante):", total_depois)
print("\nMissing por coluna (antes):")
print(missing_antes)
print("\nMissing por coluna (depois):")
print(missing_depois)

Coluna usada para Apgar: APGAR5
Total de registros (antes): 27028
Total de registros (depois de remover Apgar faltante): 26925

Missing por coluna (antes):
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1581
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS       51
APGAR5         103
dtype: int64

Missing por coluna (depois):
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1574
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS       50
APGAR5           0
dtype: int64


In [8]:
# 5) seu código aqui
# colunas alvo
cols_target = ['ESTCIVMAE', 'CONSULTAS']

# garantir que as colunas existem
cols_present = [c for c in cols_target if c in sinasc.columns]
if not cols_present:
    raise KeyError("Nenhuma das colunas ESTCIVMAE ou CONSULTAS foi encontrada no DataFrame")

# 1) tratar strings vazias ou só espaços como NA em todo o DataFrame
sinasc = sinasc.replace(r'^\s*$', pd.NA, regex=True)

# 2) substituir o código 9 por NA nas colunas alvo (apenas nas que existem)
for c in cols_present:
    sinasc[c] = sinasc[c].replace(9, pd.NA)

# 3) mostrar contagem de missing antes/depois apenas para verificação
# (se já executou antes, pode remover esta checagem)
missing_after = sinasc[cols_present].isna().sum()
print("Missing por coluna (após substituir 9 por NA):")
print(missing_after)


Missing por coluna (após substituir 9 por NA):
ESTCIVMAE    462
CONSULTAS     51
dtype: int64


In [9]:
# 6) Seu código aqui
# preparar: tratar strings vazias como NA (opcional, se ainda não foi feito)
sinasc = sinasc.replace(r'^\s*$', pd.NA, regex=True)

# garantir que a coluna existe
col = "QTDFILVIVO"
if col not in sinasc.columns:
    raise KeyError(f"Coluna {col} não encontrada no DataFrame")

# (opcional) converter sentinelas conhecidas para NA antes de preencher
# Ex.: se 9/99 significam "ignorado" na sua base, descomente/ajuste abaixo:
# sinasc[col] = sinasc[col].replace({9: pd.NA, 99: pd.NA})

# substituir NA por 0
sinasc[col] = sinasc[col].fillna(0)

# confirmar mudanças: contagem de missing (deve ser 0) e tipos
print(f"Missing em {col} após preenchimento:", sinasc[col].isna().sum())
print(f"Exemplos de valores em {col}:", sinasc[col].head(10).tolist())


Missing em QTDFILVIVO após preenchimento: 0
Exemplos de valores em QTDFILVIVO: [0.0, 1.0, 2.0, 0.0, 1.0, 2.0, 1.0, 0.0, 0.0, 1.0]


In [11]:
# 7) seu código aqui

# colunas de interesse
cols = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
        'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
cols = [c for c in cols if c in sinasc.columns]

# 1) normalizar strings vazias para NA
df = df.replace(r'^\s*$', pd.NA, regex=True)

# 2) tratar código 9 como NA nas colunas indicadas
for c in ['ESTCIVMAE', 'CONSULTAS']:
    if c in df.columns:
        df[c] = df[c].replace(9, pd.NA)

# 3) preencher QTDFILVIVO com 0 (se existir)
if 'QTDFILVIVO' in df.columns:
    df['QTDFILVIVO'] = pd.to_numeric(df['QTDFILVIVO'], errors='coerce').fillna(0).astype(int)

# 4) preencher as colunas restantes (categóricas) com 'NÃO PREENCHIDO'
#    preservando IDADEMAE e QTDFILVIVO como numéricas
preservar_numericas = {'IDADEMAE', 'QTDFILVIVO'}
col_para_label = [c for c in df.columns if c not in preservar_numericas]

for c in col_para_label:
    df[c] = df[c].fillna('NÃO PREENCHIDO')

# df agora contém o resultado essencial
# exibir resumo rápido
print("Linhas:", df.shape[0])
print(df.isna().sum())


Linhas: 27028
LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64


Justificativa 
• 	Escolhi marcar as variáveis categóricas faltantes com a etiqueta textual "NÃO PREENCHIDO" para:
• 	preservar a informação de que o valor estava ausente sem forçar uma categoria plausível que poderia enviesar análises descritivas e modelagem;
• 	facilitar filtros e contagens posteriores (é explícito e humano-interpretável);
• 	permitir criação de flags (colunas _imputado) que mantêm rastreabilidade das alterações.

In [15]:
# 8) seu código aqui
# detectar nome da coluna Apgar
apgar_col = "APGAR5" if "APGAR5" in sinasc.columns else ("APGARS" if "APGARS" in sinasc.columns else None)
if apgar_col is None:
    raise KeyError("Coluna APGAR5 ou APGARS não encontrada no DataFrame")

# copiar e normalizar valores numéricos (qualquer string será convertida para NaN se inválida)
df = sinasc[[apgar_col]].copy()
df[apgar_col] = pd.to_numeric(df[apgar_col], errors='coerce')

# criar função de categorização
def categoria_apgar(x):
    if pd.isna(x):
        return pd.NA
    x = int(x)
    if 8 <= x <= 10:
        return "normal (8-10)"
    if 6 <= x <= 7:
        return "asfixia leve (6-7)"
    if 4 <= x <= 5:
        return "asfixia moderada (4-5)"
    if 0 <= x <= 3:
        return "asfixia severa (0-3)"
    return pd.NA  # valores fora do intervalo esperado

# aplicar categorização
df['APGAR5_cat'] = df[apgar_col].apply(categoria_apgar)

# tabela de frequências absoluta e percentual (excluindo NA)
freq_abs = df['APGAR5_cat'].value_counts(dropna=True).rename("count")
freq_pct = df['APGAR5_cat'].value_counts(normalize=True, dropna=True).mul(100).round(2).rename("percent")

freq = pd.concat([freq_abs, freq_pct], axis=1).reindex([
    "normal (8-10)",
    "asfixia leve (6-7)",
    "asfixia moderada (4-5)",
    "asfixia severa (0-3)"
]).fillna(0).astype({"count": int, "percent": float})

# incluir contagem de faltantes de APGAR5
missing_apgar = df[apgar_col].isna().sum()

print("Frequências da categorização APGAR5:")
print(freq)
print(f"\nRegistros com APGAR5 ausente (NaN): {missing_apgar}")


Frequências da categorização APGAR5:
                        count  percent
APGAR5_cat                            
normal (8-10)           26463    98.28
asfixia leve (6-7)        320     1.19
asfixia moderada (4-5)     68     0.25
asfixia severa (0-3)       74     0.27

Registros com APGAR5 ausente (NaN): 103


In [16]:
# 9) seu código aqui
import re
import unicodedata

def to_snake_case(col_name):
    # normalizar acentos
    s = unicodedata.normalize('NFKD', str(col_name)).encode('ASCII', 'ignore').decode('ASCII')
    s = s.strip().lower()                      # trim + lowercase
    s = re.sub(r"[^\w\s]", "_", s)             # substituir pontuação por underscore
    s = re.sub(r"\s+", "_", s)                 # espaços -> underscore
    s = re.sub(r"_+", "_", s)                  # collapsar underscores repetidos
    s = s.strip("_")                           # remover underscores nas extremidades
    return s

sinasc.rename(columns=to_snake_case, inplace=True)

# verificação rápida
print("Novos nomes de colunas (exemplo):")
print(sinasc.columns.tolist()[:50])


Novos nomes de colunas (exemplo):
['origem', 'codestab', 'codmunnasc', 'locnasc', 'idademae', 'estcivmae', 'escmae', 'codocupmae', 'qtdfilvivo', 'qtdfilmort', 'codmunres', 'gestacao', 'gravidez', 'parto', 'consultas', 'dtnasc', 'horanasc', 'sexo', 'apgar1', 'apgar5', 'racacor', 'peso', 'idanomal', 'dtcadastro', 'codanomal', 'numerolote', 'versaosist', 'dtrecebim', 'difdata', 'dtrecoriga', 'naturalmae', 'codmunnatu', 'codufnatu', 'escmae2010', 'seriescmae', 'dtnascmae', 'racacormae', 'qtdgestant', 'qtdpartnor', 'qtdpartces', 'idadepai', 'dtultmenst', 'semagestac', 'tpmetestim', 'consprenat', 'mesprenat', 'tpapresent', 'sttrabpart', 'stcesparto', 'tpnascassi']
